In [2]:
def ngram(s,num):
    tmp0 = len(s) - num + 1
    r=[]
    for i in range(tmp0):
        w = s[i:i+num]
        r.append(w)
        pass
    return r

def diff_ngram(ta,tb,tnum):
    tmp1 = ngram(ta, tnum)
    tmp2 = ngram(tb, tnum)
    
    cnt = 0
    rs = []
    for i in tmp1:
        for j in tmp2:
            if i==j:
                cnt+=1
                rs.append(i)
                pass
            pass
        pass
    return cnt / len(tmp1), rs


a = '오늘 강남에서 맛있는 스파게티를 먹었다.'
b = '강남에서 먹었던 오늘의 스파게티는 맛있었다.'

# 2-gram
r2, word2 = diff_ngram(a,b,2)
print('2-gram:', r2, word2)

2-gram: 0.7619047619047619 ['오늘', '강남', '남에', '에서', '서 ', ' 맛', '맛있', '는 ', ' 스', '스파', '파게', '게티', ' 먹', '먹었', '었다', '다.']


# 마르코프체인을 이용한 문장 생성

In [4]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request  # 맞춤법 검사 용도, but 현재 막혀있음.
import os, re, json, random

In [21]:
def make_dic(words):
    dic = {}
    tmp = ['@']
    for word in words:
        tmp.append(word)
        if len(tmp)<3:
            continue
            pass
        if len(tmp)>3:
            tmp = tmp[1:]
            pass
        set_word3(dic,tmp)
        if word=='.':
            tmp=['@']
            continue
            pass
        pass
    return dic
        

def set_word3(dic, tmp_dic):
    w1, w2, w3 = tmp_dic
    # dic 구조에서 key값이 존재하는지 조사
    if not w1 in dic:
        dic[w1]={}
    if not w2 in dic[w1]:
        dic[w1][w2]={}
    if not w3 in dic[w1][w2]:
        dic[w1][w2][w3]=0
        pass
    dic[w1][w2][w3]+=1
    

def make_sentence(dic):
    ret=[]
    if not '@' in dic: return 'No dic'
    top = dic['@']
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == '.': break
        w1,w2 = w2,w3
    sen = ''.join(ret)
    return sen

def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))


toji_file = 'toji.txt'
dict_file = 'markov-toji.json'
if not os.path.exists(dict_file):
    fp = codecs.open('BEXX0014.txt', encoding='utf-16')
    soup = BeautifulSoup(fp, 'html.parser')
    body = soup.select_one('text > body')
    text = body.getText()
    text = text.replace('…', '')
    
    #형태소 분석
    
    twitter = Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        if not word[1] in ['punctuation']:
            words.append(word[0])
            pass
        if word[1] == '.':
            words.append(word[0])
            pass
        pass
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file, 'w', encoding='utf-8'))
    pass
else:
    dic=json.load(open(dict_file, 'r'))
    
    
# 문장 만들기
for i in range(3):
    s = make_sentence(dic)
    print(s)
    print('-----')
    

요즘시국에관해서도말했다.
-----
농담도잘갔지.
-----
이번문제를어떻게해결하지못하고서화를반려로택한것사람들은와글바글떠들었다.
-----
